<a href="https://colab.research.google.com/github/sbuttler/ddnv/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-learn==0.23.1 tensorflow==2.1.0  keras==2.3.1 ddop==0.6.7

In [53]:
from sklearn.utils.validation import check_array
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from ddop.datasets import load_yaz, load_bakery
from ddop.metrics import average_costs, prescriptiveness_score
from ddop.newsvendor import SampleAverageApproximationNewsvendor
from ddop.newsvendor import DecisionTreeWeightedNewsvendor
from ddop.newsvendor import RandomForestWeightedNewsvendor 
from ddop.newsvendor import KNeighborsWeightedNewsvendor
from ddop.newsvendor import LinearRegressionNewsvendor
from ddop.newsvendor import GaussianWeightedNewsvendor
from ddop.newsvendor import LinearRegressionNewsvendor
from ddop.newsvendor import DeepLearningNewsvendor
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import statistics

In [ ]:
n_features = 12

In [40]:
#TODO: Set Grids
dl = {"optimizer": ["adam"],
      "neurons": [
                  (round(0.5*n_features),round(0.5*0.5*n_features)),
                  (round(0.5*n_features),round(0.5*1*n_features)),
                  (1*n_features,round(1*0.5*n_features)),
                  (1*n_features,1*1*n_features),
                  (2*n_features,round(2*0.5*n_features)),
                  (2*n_features,2*1*n_features),
                  (3*n_features,round(3*0.5*n_features)),
                  (3*n_features,3*1*n_features)],
      "epochs": [10,100,200]}

dtw = {"max_depth":[1,2,3,4,5,6,7,8,10,15,20,25]}

rfw = {"max_depth":[None,2,4,6,8,10],
          'min_samples_split':[2,4,6,8,10],
          'n_estimators':[2,10,20]}

knnw = {'n_neighbors':[1,2,4,8,16,32]}

gkw = {'kernel_bandwidth':[1,1.2,1.4,1.6,1.8,2,2.5,3]}

In [41]:
# Define model tuples: 'model_name', model, grid
estimator_tuple_list = []
#estimator_tuple_list.append(('SAA', SampleAverageApproximationNewsvendor(),None))
estimator_tuple_list.append(('DTW', DecisionTreeWeightedNewsvendor(random_state=1),dtw))
#estimator_tuple_list.append(('RFW', RandomForestWeightedNewsvendor(random_state=1),rfw))
#estimator_tuple_list.append(('KNNW',KNeighborsWeightedNewsvendor(),knnw))
#estimator_tuple_list.append(('GKW', GaussianWeightedNewsvendor(),gkw))
#estimator_tuple_list.append(('DL', DeepLearningNewsvendor(),[dl]))
#estimator_tuple_list.append(('LR', LinearRegressionNewsvendor(),None))

--------------------------------------
# YAZ Dataset

In [42]:
data = load_yaz(one_hot_encoding=True)
X = data.data
y = data.target

In [43]:
products = y.columns.to_list()

No target scaling

In [45]:
estimators = []
results = pd.DataFrame()
for cu, co in zip([5,7.5,9],[5,2.5,1]):
#for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for product in products:
      
      X_train, X_test, y_train, y_test = train_test_split(X, y[product], train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
        
      else:
        gs = GridSearchCV(estimator, param_grid)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        estimators.append(best_estimator)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[products[i]+" AC"] = costs[i]
    for i in range(len(costs)):
      d[products[i]+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])

DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=1, cu=9, random_state=1)
DecisionTreeWeightedNewsvendor(co=1, cu=9, random_state=1)
DecisionTreeWeightedNewsvend

In [46]:
results

,SL,Model,calamari AC,fish AC,shrimp AC,chicken AC,koefte AC,lamb AC,steak AC,calamari SoP,fish SoP,shrimp SoP,chicken SoP,koefte SoP,lamb SoP,steak SoP,Average Cost,Score of Prescriptiveness
0,0.50,DTW,9.270833,9.296875,16.302083,48.072917,37.083333,46.666667,30.416667,-0.047059,-0.040816,0.073964,-0.125610,0.018608,0.017544,0.048860,28.158482,-0.007787
0,0.75,DTW,7.500000,7.955729,14.817708,33.059896,31.692708,40.052083,25.520833,0.076923,-0.006590,0.013865,0.134628,0.013377,0.005175,0.148566,22.942708,0.055135
0,0.90,DTW,4.479167,4.796875,8.890625,20.239583,17.250000,21.270833,15.854167,0.134809,0.163488,-0.044676,0.180860,0.086345,0.028082,0.191715,13.254464,0.105803


In [47]:
estimators

[DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=2, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=1, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=3, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=5, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=4, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=5, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=4, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=2, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=3, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=2, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=3, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=4, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=5, random_state=1),
 DecisionTreeW

With target scaling and 10 fold CV

In [54]:
estimators = []
results_sc = pd.DataFrame()
for cu, co in zip([5,7.5,9],[5,2.5,1]):
#for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for product in products:
      
      X_train, X_test, y_train, y_test = train_test_split(X, y[product], train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)
      
      #scale target variable
      scaler_target = StandardScaler()
      scaler_target.fit(np.array(y_train).reshape(-1, 1))
      y_train = scaler_target.transform(np.array(y_train).reshape(-1, 1))
      #y_test = scaler_target.transform(np.array(y_test).reshape(-1, 1))

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      saa_pred = scaler_target.inverse_transform(saa_pred)
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
          pred = scaler_target.inverse_transform(pred)
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
          pred = scaler_target.inverse_transform(pred)
        
      else:
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        gs = GridSearchCV(estimator, param_grid, cv=cv)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        pred = scaler_target.inverse_transform(pred)
        estimators.append(best_estimator)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[products[i]+" AC"] = costs[i]
    for i in range(len(costs)):
      d[products[i]+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results_sc = pd.concat([results_sc,df])

DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=1, cu=9, random_state=1)
DecisionTreeWeightedNewsvendor(co=1, cu=9, random_state=1)
DecisionTreeWeightedNewsvend

In [55]:
results_sc

,SL,Model,calamari AC,fish AC,shrimp AC,chicken AC,koefte AC,lamb AC,steak AC,calamari SoP,fish SoP,shrimp SoP,chicken SoP,koefte SoP,lamb SoP,steak SoP,Average Cost,Score of Prescriptiveness
0,0.50,DTW,8.776042,9.296875,16.302083,48.515625,37.083333,44.687500,29.114583,0.008824,-0.040816,0.073964,-0.135976,0.018608,0.059211,0.089577,27.682292,0.010484
0,0.75,DTW,7.500000,8.164062,14.635417,33.059896,31.692708,40.221354,23.984375,0.076923,-0.032949,0.025997,0.134628,0.013377,0.000970,0.199826,22.751116,0.059825
0,0.90,DTW,4.479167,4.942708,8.890625,20.239583,17.161458,21.270833,14.854167,0.134809,0.138056,-0.044676,0.180860,0.091034,0.028082,0.242698,13.119792,0.110123


In [56]:
estimators

[DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=3, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=1, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=3, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=5, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=4, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=4, random_state=1),
 DecisionTreeWeightedNewsvendor(co=5, cu=5, max_depth=5, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=2, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=1, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=3, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=3, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=4, random_state=1),
 DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, max_depth=5, random_state=1),
 DecisionTreeW

In [ ]:
results.to_excel("results_yaz_lag_per_product.xlsx")

In [ ]:
from google.colab import files

In [ ]:
files.download('results_yaz_lag_per_product.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-----------------------------------------------------------------
# Global Model (ausgegliedert)

In [ ]:
estimators = []
results = pd.DataFrame()
for cu, co in zip([5, 7.5, 9],[5, 2.5, 1]):
  saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
  for estimator_tuple in estimator_tuple_list:
    estimator_name = estimator_tuple[0]
    param_grid = estimator_tuple[2]
    estimator = estimator_tuple[1]
    estimator.set_params(cu=cu,co=co)
    print(estimator)
    if param_grid == None:
      if estimator_name=="SAA":
        pred = estimator.fit(y_train).predict(X_test.shape[0])
      else:
        pred = estimator.fit(X_train,y_train).predict(X_test)
      
    else:
      gs = GridSearchCV(estimator, param_grid)
      gs.fit(X_train,y_train)
      best_estimator = gs.best_estimator_
      pred = best_estimator.predict(X_test)
      estimators.append(best_estimator)
      
    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    avg_cost = average_costs(y_test,pred,cu,co, multioutput="raw_values")
    p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="raw_values")
    for i in range(avg_cost.size):
      d[products[i]+" AC"] = avg_cost[i]
    for i in range(avg_cost.size):
      d[products[i]+" SoP"] = p_score[i]
    avg_cost = average_costs(y_test,pred,cu,co, multioutput="uniform_average")
    d["Average Cost"] = avg_cost
    p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
    d["Score of Prescriptiveness"] = p_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])


-----------------------------------------
# Bakery Dataset

In [57]:
bakery = load_bakery(one_hot_encoding=True)
X = bakery.data
y = bakery.target

In [58]:
X_grouped = X.groupby(['product', 'store'])

In [59]:
groups = list(X_grouped.groups.keys())

In [ ]:
import statistics
estimators = []
results = pd.DataFrame()
for cu, co in zip([5,7.5,9],[5,2.5,1]):
#for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for group in groups:

      X_temp = X_grouped.get_group(group)
      y_temp = y.iloc[X_temp.index.values.tolist()]

      X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)

      #scale target variable
      scaler_target = StandardScaler()
      scaler_target.fit(np.array(y_train).reshape(-1, 1))
      y_train = scaler_target.transform(np.array(y_train).reshape(-1, 1))

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      saa_pred = scaler_target.inverse_transform(saa_pred)
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
          pred = scaler_target.inverse_transform(pred)
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
          pred = scaler_target.inverse_transform(pred)
        
      else:
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        gs = GridSearchCV(estimator, param_grid, cv=cv)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        pred = scaler_target.inverse_transform(pred)
        estimators.append(best_estimator)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[str(groups[i])+" AC"] = costs[i]
    for i in range(len(costs)):
      d[str(groups[i])+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])

DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=

In [ ]:
results

In [ ]:
estimators